In [2]:
import boto3
import json
import os
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from IPython.display import Audio

In [3]:
bedrock = boto3.client('bedrock-runtime', region_name='ap-northeast-2')

In [4]:
model_id = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
# model_id = 'anthropic.claude-3-haiku-20240307-v1:0'

In [5]:
system_prompt = '''
[role]
- 당신은 주어진 장소의 정보를 읽고 그 장소를 여행하는 사람에게 정보를 정리하여 전달하는 아나운서입니다.
- 정보는 한국어로 번역하여 전달합니다.
- 장소의 정보는 여행하는 사람이 흥미있어할 만한 내용을 위주로 정리합니다.
- 흥미있는 내용이 있다면 꼭 포함해주세요.

[current_year]
2024

[behavior_guidelines
- 정확한 정보만 제공하며, 문서에 없는 내용을 상상하여 답변하지 않습니다.
- '문서에에 따르면'과 같이 문서를 참고하는 것을 티내지 마세요.
- 사용자에게 바로 설명할 수 있도록 답변하세요.
- 아나운서의 대본처럼 바로 읽을 수 있게 답변하세요
'''

In [209]:
driver = webdriver.Chrome()

In [210]:
driver.get('https://images.google.com/')

In [211]:
driver.find_element(By.CSS_SELECTOR, "div[aria-label='이미지로 검색']").click()

In [212]:
drag_n_drop_area = driver.find_element(By.CSS_SELECTOR, "input[type='file']")

In [213]:
drag_n_drop_area.send_keys(os.path.abspath('./역삼 센터필드.jpeg'))

In [214]:
try:
    google_search = driver.find_element(By.CSS_SELECTOR, "div.wNPKTe")
    google_search.click()
except:
    None

In [215]:
driver.switch_to.window(driver.window_handles[-1])

In [216]:
wiki = driver.find_element(By.CSS_SELECTOR, "div[data-attrid='description']")

In [217]:
wiki_url = wiki.find_element(By.CSS_SELECTOR, 'a[href]').get_attribute('href')

In [218]:
response = requests.get(wiki_url)

In [219]:
soup = BeautifulSoup(response.text)

In [221]:
content = soup.find('div', {"id":"mw-content-text"}).text.strip()

In [223]:
messages =  [{"role": "user", "content": content}]

In [224]:
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 8192,
        "system": system_prompt,
        "messages": messages
    }  
)

In [225]:
response = bedrock.invoke_model(body=body, modelId=model_id)

In [226]:
response_body = json.loads(response.get('body').read())

In [227]:
script = response_body['content'][0]['text']

In [228]:
script = re.sub('\n+', '\n', script)

In [229]:
print(script)

안녕하세요, 센터필드에 대해 간단히 소개해 드리겠습니다.
센터필드는 서울 강남구 테헤란로에 위치한 대규모 복합 시설입니다. 2020년 11월에 완공된 이 건물은 웨스트타워와 이스트타워 두 개의 158m 높이 쌍둥이 빌딩으로 구성되어 있습니다.
주요 특징을 살펴보면:
- 웨스트타워는 36층, 이스트타워는 35층 규모입니다.
- 업무시설, 호텔, 판매시설 등 다양한 용도로 사용됩니다. 
- 특히 웨스트타워 29-36층에는 5성급 호텔인 '조선팰리스 서울 강남'이 들어서 있습니다.
- 건물 외관은 알루미늄 버티컬 핀과 유리를 사용해 현대적이고 세련된 디자인을 자랑합니다.
- 지하 7층, 지상 36층 규모로 연면적이 약 24만㎡에 달하는 대형 건축물입니다.
센터필드는 강남의 새로운 랜드마크로, 비즈니스와 문화, 숙박이 어우러진 복합시설로 주목받고 있습니다. 테헤란로의 중심에 위치해 접근성이 뛰어나고, 최신 시설을 갖춘 오피스와 고급 호텔 등으로 많은 사람들이 찾는 곳입니다.


In [118]:
polly_client = boto3.client('polly', region_name="ap-northeast-2")

In [126]:
# polly_client.synthesize_speech(
#     Text=script,
#     OutputFormat="mp3",
#     VoiceId="Seoyeon"
# )

In [129]:
Audio('speech_20241104145708345.mp3')

#### 근처 맛집

In [230]:
map_area = driver.find_element(By.CSS_SELECTOR, 'g-img#lu_map').find_element(By.XPATH, '..')
map_src = map_area.get_attribute('data-url')

In [150]:
map_url = map_src.split('maps/')[0] + 'maps/search/음식점' + map_src.split('maps/')[1]

In [151]:
driver.get(map_url)

------

#### url로 검색

In [231]:
image_url = 'https://search.pstatic.net/sunny/?src=https%3A%2F%2Fp1.pxfuel.com%2Fpreview%2F32%2F834%2F438%2Fparis-tour-eifel-tower-tourism.jpg&type=sc960_832'

In [232]:
driver.get('https://images.google.com/')

In [233]:
driver.find_element(By.CSS_SELECTOR, "div[aria-label='이미지로 검색']").click()

In [234]:
url_input = driver.find_element(By.CSS_SELECTOR, "input[placeholder='이미지 링크 붙여넣기']")

In [235]:
url_input.send_keys(image_url)

In [236]:
driver.find_element(By.CSS_SELECTOR, "div.PXT6cd").find_element(By.CSS_SELECTOR, "div[role='button']").click()

In [237]:
try:
    google_search = driver.find_element(By.CSS_SELECTOR, "div.wNPKTe")
    google_search.click()
except:
    None

In [238]:
driver.switch_to.window(driver.window_handles[-1])

In [239]:
wiki = driver.find_element(By.CSS_SELECTOR, "div[data-attrid='description']")

In [240]:
wiki_url = wiki.find_element(By.CSS_SELECTOR, 'a[href]').get_attribute('href')

In [241]:
response = requests.get(wiki_url)

In [242]:
soup = BeautifulSoup(response.text)

In [243]:
content = soup.find('div', {"id":"mw-content-text"}).text.strip()

In [244]:
messages =  [{"role": "user", "content": content}]

In [245]:
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 8192,
        "system": system_prompt,
        "messages": messages
    }  
)

In [246]:
response = bedrock.invoke_model(body=body, modelId=model_id)

In [247]:
response_body = json.loads(response.get('body').read())

In [248]:
script = response_body['content'][0]['text']

In [249]:
script = re.sub('\n+', '\n', script)

In [250]:
print(script)

에펠탑에 대한 주요 정보를 요약하면 다음과 같습니다:
- 프랑스 파리의 상징적인 철탑으로, 1889년 파리 만국박람회를 위해 건설되었습니다.
- 귀스타브 에펠이 설계했으며, 높이는 324m입니다. 
- 처음에는 20년 후 철거 예정이었지만, 무선 통신 안테나로 활용되면서 존속하게 되었습니다.
- 초기에는 '흉물'이라는 비난을 받았지만, 현재는 파리의 대표적인 랜드마크이자 관광명소가 되었습니다.
- 3층 구조로 되어 있으며, 1층과 2층에는 전망대와 레스토랑이 있습니다. 
- 7년마다 도색 작업을 하며, 야간 조명으로 아름다운 야경을 연출합니다.
- 매년 수백만 명의 관광객이 방문하는 세계적인 관광 명소입니다.
- 영화나 문화 콘텐츠에 자주 등장하며 파리와 프랑스를 상징하는 아이콘이 되었습니다.
에펠탑은 독특한 디자인과 역사적 의미, 그리고 파리의 상징성으로 인해 전 세계인의 사랑을 받는 건축물이 되었습니다.


In [185]:
Audio('speech_20241104153845182.mp3')

------

#### 번역 테스트

In [196]:
system_prompt = '''
[role]
- 당신은 주어진 장소의 정보를 읽고 그 장소를 여행하는 사람에게 정보를 정리하여 전달하는 아나운서입니다.
- 정보는 스페인어로 번역하여 전달합니다.
- 장소의 정보는 여행하는 사람이 흥미있어할 만한 내용을 위주로 정리합니다.
- 흥미있는 내용이 있다면 꼭 포함해주세요.

[current_year]
2024

[behavior_guidelines
- 정확한 정보만 제공하며, 문서에 없는 내용을 상상하여 답변하지 않습니다.
- '문서에에 따르면'과 같이 문서를 참고하는 것을 티내지 마세요.
- 사용자에게 바로 설명할 수 있도록 답변하세요.
- 아나운서의 대본처럼 바로 읽을 수 있게 답변하세요
'''

In [197]:
messages =  [{"role": "user", "content": content}]

In [198]:
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 8192,
        "system": system_prompt,
        "messages": messages
    }  
)

In [199]:
response = bedrock.invoke_model(body=body, modelId=model_id)

In [200]:
response_body = json.loads(response.get('body').read())

In [201]:
script = response_body['content'][0]['text']

In [202]:
script = re.sub('\n+', '\n', script)

In [203]:
print(script)

Aquí está un resumen en español de la información sobre la Torre Eiffel:
"Estimados viajeros:
Hoy les hablaré sobre uno de los monumentos más emblemáticos de París y del mundo: la Torre Eiffel. 
Esta impresionante torre de hierro fue construida entre 1887 y 1889 para la Exposición Universal de París, que celebraba el centenario de la Revolución Francesa. Fue diseñada por el ingeniero Gustave Eiffel, de quien toma su nombre.
Con sus 324 metros de altura, la Torre Eiffel fue la estructura más alta del mundo hasta 1930. Está ubicada en el Campo de Marte, a orillas del río Sena. 
Algunos datos interesantes:
- Se utilizaron más de 18,000 piezas de hierro y 2.5 millones de remaches en su construcción.
- La torre pesa unas 10,000 toneladas en total.
- Se balancea unos 6-7 cm con el viento.
- Se repinta cada 7 años, utilizando 60 toneladas de pintura.
- Recibe alrededor de 7 millones de visitantes al año.
La Torre Eiffel tiene 3 niveles accesibles al público:
- El primer piso, a 57 metros de a

In [204]:
Audio('speech_20241105003059501.mp3')

In [27]:
with open('Team9-SampyeongLuckyStar 1293228564f18000a165e95f0d04de41.md', 'r') as f:
    md = f.read()

In [28]:
print(md)

# Team9-SampyeongLuckyStar

## 구현 애플리케이션 개요

### 프로젝트 이름

- 여행찬호

### 프로젝트 배경

- 여행지에는 다양한 언어와 문화의 관광객들이 찾는다. 유명한 관광지는 각 언어로 작성된 가이드가 있지만, 그렇지 않은 관광지도 많다.
- 생성형 AI를 활용해 랜드마크, 관광지 등의 정보를 사용자의 언어로 제공받고 주변 맛집 등의 정보를 제공받는다.

### 해결 아이디어

- 랜드마크나 맛집 이미지를 입력하여 정보를 검색한다.
- 검색된 관광지의 정보를 LLM에 전달하여 가이드를 생성한다.
- 생성된 가이드는 TTS를 이용하여 음성으로 변환한다.
- 검색된 관광지 주변의 음식점을 찾는다.
- 음식점의 리뷰를 LLM으로 분석하고, 경로를 제공한다.

### 액션아이템

- 웹 스크래핑
    - 이미지 검색 구현
        - [x]  이미지를 직접 올려서 구현 → Selenium
        - [x]  이미지 url을 올려서 구현
            - url 생성 : imgur, github, notion
            - 검색 → Selenium, 서드파티 api
    - 텍스트 검색
        - [ ]  메인 페이지 진입점 시나리오
    - 주변 맛집 검색
        - [ ]  검색된 장소 → 주변 맛집 찾는 시나리오(구글 지도)
            - url 패턴을 찾아서 url을 호출하여 스크래핑
            - 셀레니움으로 스크래핑
        - [ ]  맛집 검색 이후 시나리오
            - 리뷰 요약(LLM, Claude)
            - 길안내(url)
    - 장소 정보 제공 사이트
        - [ ]  위키백과
        - [ ]  나무위키
        - [ ]  트립어드바이저 등 제 3의 사이트
- 음식점, 미술품으로 확장 가능한지 Test
    - [ ]  음식점 간판
    - [ ]  미술품(조각상, 그림)
-

In [29]:
system_prompt = '''
주어진 마크다운 문서를 읽고 github README.md를 작성해주세요
'''

In [30]:
messages =  [{"role": "user", "content": md}]

In [31]:
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 8192,
        "system": system_prompt,
        "messages": messages
    }  
)

In [32]:
response = bedrock.invoke_model(body=body, modelId=model_id)

In [33]:
response_body = json.loads(response.get('body').read())

In [34]:
print(response_body['content'][0]['text'])

# Team9-SampyeongLuckyStar

## 여행찬호 - AI 기반 여행 가이드 애플리케이션

### 프로젝트 개요

여행찬호는 생성형 AI를 활용하여 다국어 여행 가이드 서비스를 제공하는 애플리케이션입니다. 사용자가 랜드마크나 관광지의 이미지를 입력하면, AI가 해당 장소에 대한 정보를 분석하고 사용자의 언어로 가이드를 생성합니다. 또한 주변 맛집 정보와 경로 안내 기능을 제공하여 여행자들에게 편리한 경험을 선사합니다.

### 주요 기능

1. 이미지 기반 장소 인식
2. AI 생성 다국어 가이드
3. 음성 변환 가이드 (TTS)
4. 주변 맛집 검색 및 리뷰 분석
5. 경로 안내

### 기술 스택

- 웹 스크래핑: Selenium, BeautifulSoup
- AI/ML: Amazon Bedrock, LLM
- 프론트엔드: Streamlit
- 기타: Google Maps API

### 개발 현황

- [x] 이미지 검색 구현 (직접 업로드 및 URL 입력)
- [ ] 텍스트 기반 검색
- [ ] 주변 맛집 검색 및 리뷰 분석
- [ ] 장소 정보 제공 사이트 스크래핑 (위키백과, 나무위키, 트립어드바이저)
- [ ] Streamlit을 이용한 웹 인터페이스 구현
- [ ] 음식점 간판 및 미술품 인식 확장

### 향후 계획

- 사용자 플로우 및 시나리오 개선
- 다양한 언어 지원 확대
- 모바일 앱 버전 개발
- 사용자 피드백 기반 기능 개선

### 팀원

- 스테디: 사용자 플로우 및 시나리오 개발
- 단톤: 웹 스크래핑 POC
- 제인: 장소 정보 제공 사이트 스크래핑
- 이든: 서비스 진입점 시나리오 및 시스템 프롬프트 POC
- 투나: Streamlit 페이지 구현 및 백업 시나리오 개발

### 참고 자료

- [Amazon Bedrock 튜토리얼](https://www.notion.so/Amazon-Bedrock-1193228564f1804b8c5af2c60d79f35c?pvs=21)
- [해커톤 오리엔테이션 영상](ht